In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

In [ ]:
def soupme(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [ ]:
scores = list()
season_end = 2019
season_start = 1967

season_type = ["", "_playoffs"]

for season in range(season_start, season_end+1):
    url = "https://www.hockey-reference.com/leagues/NHL_{}_games.html".format(season)
    soup = soupme(url)
    
    for x in season_type:
        try:
            idname = "div_games" + x
            table_div = soup.find(id=idname)
            table = table_div.find_all('tr')
        except:
            continue

        for row in table[1:]:
            s1 = (season-1) % 100
            s2 = season % 100
            scores.append([("\'{:02d}-\'{:02d}".format(s1, s2))])
            
            # Define eras
            if season < 1984:
                scores[-1].append("TIE")
            elif season < 2000:
                scores[-1].append("OT")
            elif season < 2006:
                scores[-1].append("OTL")
            elif season < 2016:
                scores[-1].append("4on4-SO")
            else:
                scores[-1].append("3on3-SO")
                
            # Distinguish regular season vs. playoffs
            if x == "":
                scores[-1].append("Regular")
            else:
                scores[-1].append("Playoffs")

            for each in row:
                cellvalue = each.text.strip()
                cellvalue = cellvalue.replace(',', '').replace('\n', '')
                cellvalue = cellvalue.replace('Mighty Ducks of Anaheim', 'Anaheim Ducks')
                cellvalue = cellvalue.replace('Phoenix Coyotes', 'Arizona Coyotes')
                scores[-1].append(cellvalue)

df = pd.DataFrame(scores)
df.columns = ['season', 'era', 's_type', 'date', 'v_team', 'v_goals', 'h_team', 'h_goals', 'ot_so', 'attendance', 'log', 'notes']

# remove "blank" rows (postponed games)
df = df[df["v_goals"]!='']
# remove cancelled game
df = df[df["date"] != '1988-05-24']

# convert columns to proper formats (dates, goals, attendance)
df.date = pd.to_datetime(df.date)
df.v_goals = df.v_goals.astype(int)
df.h_goals = df.h_goals.astype(int)
df.attendance = pd.to_numeric(df.attendance, errors='coerce')
df['attendance'].fillna(0, inplace=True)
df.attendance = df.attendance.astype(int)

# Prepare a "max" column for sorting purposes
df["max"] = df[["v_goals", "h_goals"]].max(axis=1)

# Prepare a "final score" column
df.loc[df.v_goals > df.h_goals, 'final_score'] = df.v_goals.astype(str) + '-' + df.h_goals.astype(str) + '⠀'
df.loc[df.v_goals <= df.h_goals, 'final_score'] = df.h_goals.astype(str) + '-' + df.v_goals.astype(str) + '⠀'

# Prepare a "winner" column for home/visitor
df.loc[df.v_goals > df.h_goals, 'winner'] = 'Visitor'
df.loc[df.v_goals < df.h_goals, 'winner'] = 'Home'
df.loc[df.v_goals == df.h_goals, 'winner'] = 'Tie'

# Add "REG" and "TIE" to type, in addition to "SO" and "OT"
df.loc[df.v_goals == df.h_goals, 'ot_so'] = 'TIE'
df.loc[df.ot_so == '', 'type'] = 'REG'
df.loc[df.ot_so != '', 'type'] = df.ot_so

# add "goals" columns (one integer, one for axis)
df["goals"] = df.v_goals + df.h_goals
df["goals_axis"] = df["goals"].astype(str) + 'G'

# Drop unneeded columns
try:
#     df.drop(["ot_so", "log", "notes"], axis=1, inplace=True)
    df.drop(["ot_so", "log"], axis=1, inplace=True)

except:
    pass

# Write to csv files
df.to_csv("nhl.csv", index=False)


In [ ]:
# Sort data and export to .csv
# df.sort_values(by=['goals', 'max'], inplace=True, ascending=False)
# df.to_csv("nhl_sorted.csv", index=False)

In [ ]:
# df[df["date"] == '2014-01-21']
# df.attendance = df.attendance.astype(int)
# df[df["attendance"] == '\n']
# df.loc[df.attendance == '', df.attendance] = '0'

In [ ]:
df.dtypes

In [ ]:
df